# Testing New Rankings Data
https://github.com/JeffSackmann/tennis_atp
The player file columns are player_id, first_name, last_name, hand, birth_date, country_code.
The columns for the ranking files are ranking_date, ranking, player_id, ranking_points.
E:\Development\0117-DaiAnalysis\gitHub_atp_project\tennis_atp\rankings

In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import dask.dataframe as dd
from MutiprocessDataframe import callfunction

basefolder=r'E:\Development\0117-DaiAnalysis\gitHub_atp_project\tennis_atp\rankings'
print ("Calling MultiProcess")
ranklist = callfunction(basefolder)
ranklist.shape

Calling MultiProcess


(2499548, 4)

In [2]:
#ranklist=ranklist.rename({'0':'RankDate','1':'Rank','3':'PlayerID','4':'RankPoints;'})
ranklist.head()

,rank_date,rank,player_id,rank_points
0,19900101,1,100656,2913
1,19900101,2,101414,2279
2,19900101,3,101222,2111
3,19900101,4,100763,1398
4,19900101,5,100581,1354


In [3]:
file=r"E:\Development\0117-DaiAnalysis\gitHub_atp_project\tennis_atp\atp_players2.csv"
players=pd.read_csv(file,encoding = "ISO-8859-1",header=None,names=['player_id', 'first_name', 'last_name', 'hand', 'birth_date', 'country_code'])

In [4]:
print(players.shape)
players.head()

(45684, 6)


,player_id,first_name,last_name,hand,birth_date,country_code
0,100001,Gardnar,Mulloy,R,19131122.0,USA
1,100002,Pancho,Segura,R,19210620.0,ECU
2,100003,Frank,Sedgman,R,19271002.0,AUS
3,100004,Giuseppe,Merlo,R,19271011.0,ITA
4,100005,Richard Pancho,Gonzales,R,19280509.0,USA


In [5]:
dupes=players[players.duplicated(['first_name', 'last_name'], keep=False)]
print(dupes.shape)

(503, 6)



## Drop Dupes with NaN
#df.dropna()     #drop all rows that have any NaN values

In [6]:
dupes = dupes.dropna()
print(dupes.shape)
players=players.drop_duplicates(['first_name', 'last_name'])
print(players.shape)
players=players.append(dupes, ignore_index=True)
print(players.shape)

(115, 6)
(45406, 6)
(45521, 6)


In [7]:
groupeddupes=dupes.groupby('last_name')
#print(dupes.shape)
for key,item in groupeddupes:
    print(key)
    print(groupeddupes.get_group(key))

Adams
       player_id first_name last_name hand  birth_date country_code
38148     138112      Henry     Adams    U  19960621.0          USA
Al Baloushi
       player_id     first_name    last_name hand  birth_date country_code
8982      108996  Mahmoud Nader  Al Baloushi    U  19800603.0          UAE
42511     142475  Mahmoud Nader  Al Baloushi    U  19800630.0          UAE
Alonso
       player_id first_name last_name hand  birth_date country_code
3030      103031     Julian    Alonso    R  19770802.0          ESP
31399     131363     Julian    Alonso    R  19770802.0          ESP
Alvarez
      player_id first_name last_name hand  birth_date country_code
9819     109782    William   Alvarez    R  19341215.0          COL
Antelo
       player_id first_name last_name hand  birth_date country_code
17456     117420       Jose    Antelo    U  19820703.0          BOL
Aquino
       player_id first_name last_name hand  birth_date country_code
25452     125416   Mauricio    Aquino    U  196803

## Merging Players and Ranks into Results
#Doing this for all oponents and players

In [8]:
play = players[['first_name','last_name','player_id']].copy()
#ranklist=ranklist.rename({'0':'RankDate','1':'Rank','3':'PlayerID','4':'RankPoints;'})
p_players=play.rename(columns={'first_name':'p_first_name', 'last_name':'p_last_name','player_id':'p_player_id'})

In [9]:
file="C:\\Users\\amac\\Documents\\GoogleDrive\\Washu\\01-17-Dai\\singles_results_Katy.dta"
results = pd.read_stata(file, convert_dates=True, convert_categoricals=True, encoding=None, index=None, convert_missing=False, preserve_dtypes=True, columns=None, order_categoricals=True, chunksize=None, iterator=False)
#list(results)
results['date'] = pd.to_datetime(results['date'])
results = results.merge(p_players,on=['p_first_name','p_last_name'],how='left')

In [10]:
o_players=play.rename(columns={'first_name':'o_first_name', 'last_name':'o_last_name','player_id':'o_player_id'})
results = results.merge(o_players,on=['o_first_name','o_last_name'],how='left')
results.shape

(62183, 44)

In [11]:

ranklist['rank_date']=pd.to_datetime(ranklist['rank_date'],format="%Y%m%d")
ranklist.tail()

,rank_date,rank,player_id,rank_points
2499543,1989-10-23,952,136381,0
2499544,1989-10-23,952,136381,0
2499545,1989-12-11,919,108831,0
2499546,1989-12-11,950,136381,0
2499547,1989-12-11,950,136381,0


In [12]:
ranklist.head()

,rank_date,rank,player_id,rank_points
0,1990-01-01,1,100656,2913
1,1990-01-01,2,101414,2279
2,1990-01-01,3,101222,2111
3,1990-01-01,4,100763,1398
4,1990-01-01,5,100581,1354


In [13]:
ranklist = ranklist[['rank_date','player_id','rank']].copy()
ranklist['date']=ranklist['rank_date']
p_ranklist = ranklist.rename(columns={'rank_date':'p_rank_date','player_id':'p_player_id','rank':'p_rank'})
p_ranklist.shape

(2499548, 4)

In [14]:
results.head()

,winloss,p_last_name,p_first_name,p_natl,o_last_name,o_first_name,o_natl,year,tournament,date,...,retgamo,retgamp,rgamply,sgamply,ssply,ssroppr,ssrwon,sswon,p_player_id,o_player_id
0,L,Youzhny,Mikhail,RUS,Kiefer,Nicolas,GER,2007,ATP Masters Series Madrid,2007-10-15,...,8.0,8.0,8.0,8.0,19.0,16.0,9.0,9.0,104022,103017.0
1,W,Lopez,Feliciano,ESP,Koubek,Stefan,AUT,2007,ATP Masters Series Madrid,2007-10-15,...,10.0,9.0,9.0,10.0,29.0,30.0,12.0,18.0,103852,102905.0
2,L,Safin,Marat,RUS,Karlovic,Ivo,CRO,2007,ATP Masters Series Madrid,2007-10-15,...,9.0,10.0,10.0,9.0,21.0,15.0,8.0,12.0,103498,103333.0
3,L,Ljubicic,Ivan,CRO,Koubek,Stefan,AUT,2007,ATP Masters Series Madrid,2007-10-15,...,16.0,17.0,17.0,16.0,32.0,39.0,13.0,15.0,103344,102905.0
4,L,Nieminen,Jarkko,FIN,Koubek,Stefan,AUT,2007,ATP Masters Series Madrid,2007-10-15,...,14.0,14.0,14.0,14.0,31.0,36.0,18.0,16.0,103813,102905.0


In [15]:
results1=results.merge(p_ranklist,on=['p_player_id','date'],how='inner')

In [16]:
results1.head()

,winloss,p_last_name,p_first_name,p_natl,o_last_name,o_first_name,o_natl,year,tournament,date,...,rgamply,sgamply,ssply,ssroppr,ssrwon,sswon,p_player_id,o_player_id,p_rank_date,p_rank
0,L,Youzhny,Mikhail,RUS,Kiefer,Nicolas,GER,2007,ATP Masters Series Madrid,2007-10-15,...,8.0,8.0,19.0,16.0,9.0,9.0,104022,103017.0,2007-10-15,18
1,W,Lopez,Feliciano,ESP,Koubek,Stefan,AUT,2007,ATP Masters Series Madrid,2007-10-15,...,9.0,10.0,29.0,30.0,12.0,18.0,103852,102905.0,2007-10-15,42
2,L,Lopez,Feliciano,ESP,Federer,Roger,SUI,2007,ATP Masters Series Madrid,2007-10-15,...,11.0,11.0,38.0,23.0,5.0,17.0,103852,103819.0,2007-10-15,42
3,W,Lopez,Feliciano,ESP,Ferrer,David,ESP,2007,ATP Masters Series Madrid,2007-10-15,...,12.0,12.0,43.0,35.0,16.0,27.0,103852,103970.0,2007-10-15,42
4,W,Lopez,Feliciano,ESP,Volandri,Filippo,ITA,2007,ATP Masters Series Madrid,2007-10-15,...,8.0,9.0,20.0,24.0,14.0,7.0,103852,103835.0,2007-10-15,42


In [17]:
o_ranklist = ranklist.rename(columns={'rank_date':'o_rank_date','player_id':'o_player_id','rank':'o_rank'})
results2=results1.merge(o_ranklist,on=['o_player_id','date'],how='inner')

In [18]:
results2.shape

(60315, 48)

In [19]:
results3 = results2.dropna(subset=['o_rank','p_rank'])
results3.shape

(60315, 48)

In [20]:
results3.head()

,winloss,p_last_name,p_first_name,p_natl,o_last_name,o_first_name,o_natl,year,tournament,date,...,ssply,ssroppr,ssrwon,sswon,p_player_id,o_player_id,p_rank_date,p_rank,o_rank_date,o_rank
0,L,Youzhny,Mikhail,RUS,Kiefer,Nicolas,GER,2007,ATP Masters Series Madrid,2007-10-15,...,19.0,16.0,9.0,9.0,104022,103017.0,2007-10-15,18,2007-10-15,112
1,W,Federer,Roger,SUI,Kiefer,Nicolas,GER,2007,ATP Masters Series Madrid,2007-10-15,...,17.0,35.0,16.0,10.0,103819,103017.0,2007-10-15,1,2007-10-15,112
2,L,Gonzalez,Fernando,CHI,Kiefer,Nicolas,GER,2007,ATP Masters Series Madrid,2007-10-15,...,21.0,26.0,10.0,9.0,103602,103017.0,2007-10-15,6,2007-10-15,112
3,W,Lopez,Feliciano,ESP,Koubek,Stefan,AUT,2007,ATP Masters Series Madrid,2007-10-15,...,29.0,30.0,12.0,18.0,103852,102905.0,2007-10-15,42,2007-10-15,68
4,L,Ljubicic,Ivan,CRO,Koubek,Stefan,AUT,2007,ATP Masters Series Madrid,2007-10-15,...,32.0,39.0,13.0,15.0,103344,102905.0,2007-10-15,13,2007-10-15,68


In [21]:
results3.to_stata('no_date_fudging_0129.dta',convert_dates={'date':'tc','p_rank_date':'tc','o_rank_date':'tc'})

In [22]:
results.dtypes

winloss                   object
p_last_name               object
p_first_name              object
p_natl                    object
o_last_name               object
o_first_name              object
o_natl                    object
year                       int16
tournament                object
date              datetime64[ns]
tournamenttype            object
round                       int8
set1p                    float64
set1o                    float64
set2p                    float64
set2o                    float64
set3p                    float64
set3o                    float64
set4p                    float64
set4o                    float64
set5p                    float64
set5o                    float64
rsn                       object
aces                        int8
bpface                   float64
bplst                    float64
bpoppr                   float64
bpwon                    float64
dblfalt                  float64
fsply                    float64
fsroppr   

In [23]:
res=results[results['o_player_id'].isnull()]
# results['p_player_id']=results['p_player_id'].astype(int)
# results.dtype
cnt=0
nares=res.groupby(['o_last_name','o_first_name'])
for key,val in nares:
    print(key)
    cnt+=len(nares.get_group(key))

('Albert-Ferrando', 'Mariano')
('Anzari ', 'Tomas')
('Aparisi', 'Jose-Luis')
('Arnold Ker', 'Lucas')
('Bachelot', 'Jean-Francois')
('Bahrouzyan', 'Omar ')
('Cunha-Silva', 'Joao')
('Davis', 'Martin')
('DeVries', 'Steve')
('Gabashvili', 'Teimuraz')
('Ghareeb', 'Mohammed')
('Imaz-Ruiz', 'Jose')
('Jun', 'Woong-Sun')
('Kim', 'Bong-Soo')
('Kim', 'Chi-Wan')
('Kirillov', 'Evgeniy')
('Lee', 'Hyung-Taik')
('Lu', 'Yen-Hsun')
('Middleton', 'T.J.')
("N'Goran", 'Claude')
("N'Goran", 'Clement')
("O'Brien", 'Alex')
('Roger-Vasselin', 'Edouard')
('Wang', 'Yeu-Tzuoo')
('Xia', 'Jia-Ping')
('Zhu', 'Ben-Qiang')
('Zingman', 'Andres')
('de Jager', 'John-Laffnie')
('van Lottum', 'John')
('van Scheppingen', 'Dennis')


In [24]:
cnt

576

In [25]:
res=results[results['p_player_id'].isnull()]
# results['p_player_id']=results['p_player_id'].astype(int)
# results.dtype
res.shape
#nares=res.groupby(['p_last_name','p_first_name'])
# for key,val in nares:
#     print(key)

(0, 44)

In [26]:
res=results[results['o_player_id'].isnull()]
# results['p_player_id']=results['p_player_id'].astype(int)
# results.dtype
res.shape

(576, 44)

In [27]:
results=results.dropna(subset=['o_player_id','p_player_id'])

# Looking at non matching dates
There are 6k records where dates dont match. Thats 10%.  
Perform logic finding both the ranks where the rank must be before the match AND where the rank can be on match date

In [28]:
ranklist.dtypes
#ranklist.head()

rank_date    datetime64[ns]
player_id             int64
rank                  int64
date         datetime64[ns]
dtype: object

In [29]:
ranklist['player_id']=ranklist['player_id'].astype(int)
ranklist.dtypes

rank_date    datetime64[ns]
player_id             int32
rank                  int64
date         datetime64[ns]
dtype: object

In [30]:

results['p_player_id']=results['p_player_id'].astype(int)
results['o_player_id']=results['o_player_id'].astype(int)
results.dtypes

winloss                   object
p_last_name               object
p_first_name              object
p_natl                    object
o_last_name               object
o_first_name              object
o_natl                    object
year                       int16
tournament                object
date              datetime64[ns]
tournamenttype            object
round                       int8
set1p                    float64
set1o                    float64
set2p                    float64
set2o                    float64
set3p                    float64
set3o                    float64
set4p                    float64
set4o                    float64
set5p                    float64
set5o                    float64
rsn                       object
aces                        int8
bpface                   float64
bplst                    float64
bpoppr                   float64
bpwon                    float64
dblfalt                  float64
fsply                    float64
fsroppr   

In [31]:
#create groups
p_resultsgrouped = results.groupby('p_player_id')
o_resultsgrouped = results.groupby('o_player_id')
rankgrouped = ranklist.groupby('player_id')

# for each group
# for each playeddate
p_newdates=pd.DataFrame(columns=('p_player_id', 'date', 'p_rank_date'))
o_newdates=pd.DataFrame(columns=('o_player_id', 'date', 'o_rank_date'))

In [32]:
gr=dict(list(p_resultsgrouped))
#print (gr)
p_resultsgrouped.get_group(100113)
rankgrouped.get_group(100113)

,rank_date,player_id,rank,date
35414,1990-08-13,100113,566,1990-08-13
86199,1991-07-29,100113,1090,1991-07-29
87474,1991-08-05,100113,1099,1991-08-05
88754,1991-08-12,100113,1093,1991-08-12
90020,1991-08-19,100113,1094,1991-08-19
91291,1991-08-26,100113,1095,1991-08-26
92373,1991-09-02,100113,1095,1991-09-02
92851,1991-09-09,100113,1103,1991-09-09
94133,1991-09-16,100113,1098,1991-09-16
95422,1991-09-23,100113,1106,1991-09-23


In [33]:
p_newdates=[]
# plistgroupeddate
# p_resultsgrouped.get_group(key).p_last_name.iloc[0]

In [34]:
cnt=0
nokey=0
for key,item in p_resultsgrouped:
    print("=====Key=====",key)
    resultsgroupeddate=p_resultsgrouped.get_group(key).date
    lastname=p_resultsgrouped.get_group(key).p_last_name.iloc[0]
    print(lastname)
    try:
        plistgroupeddate=rankgrouped.get_group(key).rank_date
    except:
        print("===Key Doesnt Exist===",key)  
        
    for i,playedDate in enumerate(resultsgroupeddate):
        #print(playedDate)
        #print(plistgroupeddate)
        time_deltas = plistgroupeddate-playedDate #np.subtract(playedDate - rankDateSeries) 
        tdo=time_deltas.copy()
        #print(time_deltas) 
        time_deltas2 = time_deltas[time_deltas <= pd.Timedelta(0)]
        #print(time_deltas[:5])
        try:
            idx_closest_date = time_deltas2.argmax() #np.argmin(time_deltas)
        #set the value in ranklist to the new date        
            
#         except:
#             try:
#                 idx_closest_date = time_deltas2.argmin()
            newdate=plistgroupeddate.loc[idx_closest_date]
        except:
            print(plistgroupeddate)
            print("no Argmax")
            cnt += 1
            print(playedDate)
            newdate = "N/A"

#         #print(newdate)
#         #groupranks.iloc[idx_closest_date]['date']=newdate
#         print(time_deltas.loc[idx_closest_date])
#         #print(idx_closest_date)
#         plistdate=plist.loc[idx_closest_date]['date']
#         #print(plistdate)
#         plistdate=playedDate
        #Create new dataframe
        if playedDate != newdate or newdate=="N/A":
            p_newdates.append([key,playedDate,newdate])
        #ranklist.set_value(idx_closest_date,'date',playedDate)
        #print(plist.loc[idx_closest_date]['date'])


=====Key===== 100113
Kodes Jr.
=====Key===== 100282
Vilas
=====Key===== 100284
Connors
=====Key===== 100286
Solomon
=====Key===== 100321
Amritraj
=====Key===== 100437
Borg
=====Key===== 100529
Curren
=====Key===== 100532
Kriek
=====Key===== 100581
McEnroe
=====Key===== 100586
Teltscher
=====Key===== 100653
Gomez
=====Key===== 100656
Lendl
=====Key===== 100676
Noah
=====Key===== 100689
Mayotte
=====Key===== 100757
Jarryd
=====Key===== 100763
Gilbert
=====Key===== 100809
Aguilera
=====Key===== 100815
Pate
=====Key===== 100855
Davis
=====Key===== 100870
Van Rensburg
=====Key===== 100901
Annacone
=====Key===== 100923
Masur
=====Key===== 100936
Leconte
=====Key===== 100942
Tulasne
=====Key===== 100944
Pernfors
=====Key===== 100946
Zivojinovic
=====Key===== 101030
Prpic
=====Key===== 101036
Mecir (Sr.)
=====Key===== 101061
Arias
=====Key===== 101062
Wilander
=====Key===== 101073
Jaite
=====Key===== 101084
Hlasek
=====Key===== 101098
Forget
=====Key===== 101120
Novacek
=====Key===== 101138
Ca

In [ ]:
print 
len(p_newdates)

#so now merge this with results, where key is p_player_id
#and fill the NA's with the date

#and then do the same for the o_player

#and then do the same for where date < 0 

#and probably ignore the ones where theyr'e not ranked. need to talk to dye

922

In [ ]:
p_newdates_before=[]
cnt=0
nokey=0
for key,item in p_resultsgrouped:
    print("=====Key=====",key)
    resultsgroupeddate=p_resultsgrouped.get_group(key).date
    lastname=p_resultsgrouped.get_group(key).p_last_name.iloc[0]
    print(lastname)
    try:
        plistgroupeddate=rankgrouped.get_group(key).rank_date
    except:
        print("===Key Doesnt Exist===",key)  
        
    for i,playedDate in enumerate(resultsgroupeddate):
        #print(playedDate)
        #print(plistgroupeddate)
        time_deltas = plistgroupeddate-playedDate #np.subtract(playedDate - rankDateSeries) 
        tdo=time_deltas.copy()
        #print(time_deltas) 
        time_deltas2 = time_deltas[time_deltas < pd.Timedelta(0)]
        #print(time_deltas[:5])
        try:
            idx_closest_date = time_deltas2.argmax() #np.argmin(time_deltas)
        #set the value in ranklist to the new date        
            
#         except:
#             try:
#                 idx_closest_date = time_deltas2.argmin()
            newdate=plistgroupeddate.loc[idx_closest_date]
        except:
            print(plistgroupeddate)
            print("no Argmax")
            cnt += 1
            print(playedDate)
            newdate = "N/A"

#         #print(newdate)
#         #groupranks.iloc[idx_closest_date]['date']=newdate
#         print(time_deltas.loc[idx_closest_date])
#         #print(idx_closest_date)
#         plistdate=plist.loc[idx_closest_date]['date']
#         #print(plistdate)
#         plistdate=playedDate
        #Create new dataframe
        if playedDate != newdate or newdate=="N/A":
            p_newdates_before.append([key,playedDate,newdate])
        #ranklist.set_value(idx_closest_date,'date',playedDate)
        #print(plist.loc[idx_closest_date]['date'])


=====Key===== 100113
Kodes Jr.
=====Key===== 100282
Vilas
=====Key===== 100284
Connors
=====Key===== 100286
Solomon
=====Key===== 100321
Amritraj
=====Key===== 100437
Borg
=====Key===== 100529
Curren
=====Key===== 100532
Kriek
=====Key===== 100581
McEnroe
=====Key===== 100586
Teltscher
=====Key===== 100653
Gomez
=====Key===== 100656
Lendl
=====Key===== 100676
Noah
=====Key===== 100689
Mayotte
=====Key===== 100757
Jarryd
=====Key===== 100763
Gilbert
=====Key===== 100809
Aguilera
=====Key===== 100815
Pate
=====Key===== 100855
Davis
=====Key===== 100870
Van Rensburg
=====Key===== 100901
Annacone
=====Key===== 100923
Masur
=====Key===== 100936
Leconte
=====Key===== 100942
Tulasne
=====Key===== 100944
Pernfors
=====Key===== 100946
Zivojinovic
=====Key===== 101030
Prpic
=====Key===== 101036
Mecir (Sr.)
=====Key===== 101061
Arias
=====Key===== 101062
Wilander
=====Key===== 101073
Jaite
=====Key===== 101084
Hlasek
=====Key===== 101098
Forget
=====Key===== 101120
Novacek
=====Key===== 101138
Ca

In [ ]:
len(p_newdates_before)

In [ ]:
o_newdates=[]
cnt=0
nokey=0
for key,item in o_resultsgrouped:
    print("=====Key=====",key)
    resultsgroupeddate=o_resultsgrouped.get_group(key).date
    lastname=o_resultsgrouped.get_group(key).o_last_name.iloc[0]
    print(lastname)
    try:
        plistgroupeddate=rankgrouped.get_group(key).rank_date
    except:
        print("===Key Doesnt Exist===",key)  
        
    for i,playedDate in enumerate(resultsgroupeddate):
        #print(playedDate)
        #print(plistgroupeddate)
        time_deltas = plistgroupeddate-playedDate #np.subtract(playedDate - rankDateSeries) 
        tdo=time_deltas.copy()
        #print(time_deltas) 
        time_deltas2 = time_deltas[time_deltas <= pd.Timedelta(0)]
        #print(time_deltas[:5])
        try:
            idx_closest_date = time_deltas2.argmax() #np.argmin(time_deltas)
        #set the value in ranklist to the new date        
            
#         except:
#             try:
#                 idx_closest_date = time_deltas2.argmin()
            newdate=plistgroupeddate.loc[idx_closest_date]
        except:
            print(plistgroupeddate)
            print("no Argmax")
            cnt += 1
            print(playedDate)
            newdate = "N/A"

#         #print(newdate)
#         #groupranks.iloc[idx_closest_date]['date']=newdate
#         print(time_deltas.loc[idx_closest_date])
#         #print(idx_closest_date)
#         plistdate=plist.loc[idx_closest_date]['date']
#         #print(plistdate)
#         plistdate=playedDate
        #Create new dataframe
        if playedDate != newdate or newdate=="N/A":
            o_newdates.append([key,playedDate,newdate])
        #ranklist.set_value(idx_closest_date,'date',playedDate)
        #print(plist.loc[idx_closest_date]['date'])

In [ ]:
len(o_newdates)

In [ ]:
o_newdates_before=[]
cnt=0
nokey=0
for key,item in o_resultsgrouped:
    print("=====Key=====",key)
    resultsgroupeddate=o_resultsgrouped.get_group(key).date
    lastname=o_resultsgrouped.get_group(key).o_last_name.iloc[0]
    print(lastname)
    try:
        plistgroupeddate=rankgrouped.get_group(key).rank_date
    except:
        print("===Key Doesnt Exist===",key)  
        
    for i,playedDate in enumerate(resultsgroupeddate):
        #print(playedDate)
        #print(plistgroupeddate)
        time_deltas = plistgroupeddate-playedDate #np.subtract(playedDate - rankDateSeries) 
        tdo=time_deltas.copy()
        #print(time_deltas) 
        time_deltas2 = time_deltas[time_deltas < pd.Timedelta(0)]
        #print(time_deltas[:5])
        try:
            idx_closest_date = time_deltas2.argmax() #np.argmin(time_deltas)
        #set the value in ranklist to the new date        
            
#         except:
#             try:
#                 idx_closest_date = time_deltas2.argmin()
            newdate=plistgroupeddate.loc[idx_closest_date]
        except:
            print(plistgroupeddate)
            print("no Argmax")
            cnt += 1
            print(playedDate)
            newdate = "N/A"

#         #print(newdate)
#         #groupranks.iloc[idx_closest_date]['date']=newdate
#         print(time_deltas.loc[idx_closest_date])
#         #print(idx_closest_date)
#         plistdate=plist.loc[idx_closest_date]['date']
#         #print(plistdate)
#         plistdate=playedDate
        #Create new dataframe
        if playedDate != newdate or newdate=="N/A":
            o_newdates_before.append([key,playedDate,newdate])
        #ranklist.set_value(idx_closest_date,'date',playedDate)
        #print(plist.loc[idx_closest_date]['date'])

In [ ]:
len(o_newdates_before)

In [ ]:
o_newdates=pd.DataFrame(np.array(o_newdates_before),columns=['o_player_id','date','p_newdate_on'],dtype=['int','datetime64[ns]','datetime64[ns]'])
#df['time'] = df['time'].astype('datetime64[ns]')

In [ ]:
o_newdates.head()